In [1]:
import timeit

import pandas as pd
import plotly.express as px

from vrp.problem_instance import ProblemInstance
from vrp.vrp_solver import ExhaustiveHeuristic, NearestNeighborHeuristic, SweepAngleHeuristic, VRPSolution, VRPSolver

In [2]:
def get_opt_solution(problem_instance: ProblemInstance) -> VRPSolution:
    solver = VRPSolver(
        problem_instance,
        [ExhaustiveHeuristic()],
    )
    sol = solver.solve()
    return sol


def get_appx_solution(problem_instance: ProblemInstance) -> VRPSolution:
    solver = VRPSolver(
        problem_instance,
        [NearestNeighborHeuristic(), SweepAngleHeuristic()],
    )
    sol = solver.solve()
    return sol

In [3]:
# solve the example problem optimaly
problem_instance = ProblemInstance.from_csv(
    file_path="instances/example.csv",
    num_vehicles=3,
    vehciel_capacity=15,
)
opt_solution = get_opt_solution(problem_instance)
print(opt_solution)
opt_solution.visualize()


Path 0: Depot -> 1 -> 7 -> 4 -> 2

Path 1: Depot -> 6 -> 8 -> 9

Path 2: Depot -> 3 -> 5 -> 10

Total Cost: 62.64813890655621



In [4]:
# solve the example problem approximately
problem_instance = ProblemInstance.from_csv(
    file_path="instances/example.csv",
    num_vehicles=3,
    vehciel_capacity=15,
)
appx_solution = get_appx_solution(problem_instance)
print(appx_solution)
appx_solution.visualize()


Path 0: Depot -> 3 -> 5 -> 10

Path 1: Depot -> 6 -> 8 -> 9

Path 2: Depot -> 1 -> 7 -> 4 -> 2

Total Cost: 62.64813890655621



In [7]:
# profile runtime of approximate solution
def get_runtime(problem_instance: ProblemInstance) -> float:
    print(f"solving for instance size {len(problem_instance.locations)}")
    start = timeit.default_timer()
    solver = VRPSolver(
        problem_instance,
        [SweepAngleHeuristic()],
    )
    sol = solver.solve()
    stop = timeit.default_timer()
    return stop - start


runtimes = [(i, get_runtime(ProblemInstance.from_random(num_locations=i))) for i in list(range(10, 71, 10))]
runtimes_frame = pd.DataFrame(runtimes, columns=["num_locations", "runtime"])

fig = px.line(runtimes_frame, x="num_locations", y="runtime")
fig

solving for instance size 10
solving for instance size 20
solving for instance size 30
solving for instance size 40
solving for instance size 50
solving for instance size 60
solving for instance size 70


In [6]:
# profile optimality performance
def get_optimality_gap(problem_instance: ProblemInstance):
    print(f"solving for instance size {len(problem_instance.locations)}")
    opt_solution = get_opt_solution(problem_instance)
    appx_solution = get_appx_solution(problem_instance)
    return (opt_solution.distance - appx_solution.distance) / opt_solution.distance


optimality_gaps = [(i, get_optimality_gap(ProblemInstance.from_random(num_locations=i))) for i in [10, 15, 20]]
optimality_gaps_frame = pd.DataFrame(optimality_gaps, columns=["num_locations", "optimality_gap"])

fig = px.line(optimality_gaps_frame, x="num_locations", y="optimality_gap")
fig

solving for instance size 10
solving for instance size 15
solving for instance size 20
